In [ ]:
"""
This code sample shows Prebuilt Layout operations with the Azure AI Document Intelligence client library.
The async versions of the samples require Python 3.8 or later.

To learn more, please visit the documentation - Quickstart: Document Intelligence (formerly Form Recognizer) SDKs
https://learn.microsoft.com/azure/ai-services/document-intelligence/quickstarts/get-started-sdks-rest-api?pivots=programming-language-python
"""

from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest

"""
Remember to remove the key from your code when you're done, and never post it publicly. For production, use
secure methods to store and access your credentials. For more information, see 
https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-security?tabs=command-line%2Ccsharp#environment-variables-and-application-configuration
"""
endpoint = "https://westeurope.api.cognitive.microsoft.com/"
key = "xxx" # your key here

# sample document
#formUrl = "https://raw.githubusercontent.com/ElisabethWeigel/GPT-FR-index-sectionheadings/blob/main/Landesbauordnung.pdf"

#"https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"

document_intelligence_client  = DocumentIntelligenceClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)




In [38]:
import base64

local_file="CELEX_32013D0121_EN_TXT.pdf" #"Landesbauordnung.pdf"

with open(local_file, "rb") as f:
    analyze_request = {
        "base64Source": base64.b64encode(f.read()).decode("utf-8")
    }
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout", 
         body=analyze_request,
         output_content_format="markdown"
    )
result= poller.result()
print(result.content)

<!-- PageNumber="L 65/23" -->
<!-- PageHeader="Official Journal of the European Union" -->
<!-- PageHeader="EN" -->
<!-- PageHeader="8.3.2013" -->


# COMMISSION DECISION of 7 March 2013

on the safety requirements to be met by European standards for certain seats for children pursuant
to Directive 2001/95/EC of the European Parliament and of the Council on general product safety
(Text with EEA relevance)
(2013/121/EU)

THE EUROPEAN COMMISSION,

Having regard to the Treaty on the Functioning of the European
Union,

Having regard to Directive 2001/95/EC of the European
Parliament and of the Council of 3 December 2001 on
general product safety (1), and in particular Article 4(1)(a)
thereof,

Whereas:

(1)
Products which conform to national standards that
transpose European standards drawn up under Directive
2001/95/EC and referenced in the Official Journal of the
European Union benefit from a presumption of safety.

(2)
European standards are to be drawn up on the basis of
requirements i

In [39]:
#set title
titel=""
for i, paragraph in enumerate(result.paragraphs):
    if(paragraph.role=="title"):        
        titel= paragraph.content
        print("Titel" + titel)
        
sectionHeading=""
    #set sectionHeadings
for i, paragraph in enumerate(result.paragraphs):
    if(paragraph.role=="sectionHeading"):
        sectionHeading =paragraph.content
        print("Section Heading " +sectionHeading)
        
        
#get a list of footers
footerList = []
for i, paragraph in enumerate(result.paragraphs):
    if(paragraph.role=="footer"):
        footerList.append(paragraph.content)
        print("Footer " + paragraph.content)
        
#get a list of headers
headerList = []
for i, paragraph in enumerate(result.paragraphs):
    if(paragraph.role=="pageHeader"):
        headerList.append(paragraph.content)
        print("Header " + paragraph.content)
print("----------------------------------------")


TitelCOMMISSION DECISION of 7 March 2013
Section Heading Article 1
Section Heading Definitions
Section Heading Article 2 Safety requirements
Section Heading Article 3 Entry into force
Section Heading ANNEX
Section Heading GENERAL SAFETY REQUIREMENTS
Section Heading Chemical requirements
Section Heading Flammable properties
Section Heading Packaging
Section Heading Identification of the manufacturer and importer
Section Heading SPECIFIC SAFETY REQUIREMENTS
Section Heading 1. CHAIR-MOUNTED SEATS
Section Heading 1.1. Scope
Section Heading 1.2. Safety requirements
Section Heading Entanglement hazards
Section Heading Choking hazards
Section Heading Ingestion hazards
Section Heading Suffocation hazards
Section Heading Surfaces
Section Heading Structural integrity
Section Heading 1.3. Safety information, user manual and markings
Section Heading Safety information
Section Heading Purchase information
Section Heading User manual
Section Heading 2.1. Scope
Section Heading 2.2. Safety requirement

In [40]:
#split the content of results into chunks cut by the section headings. 

content=""
for i, paragraph in enumerate(result.paragraphs):
    if(paragraph.role=="sectionHeading"):
        content += "\n\n" + paragraph.content
    else:
        content += "\n" + paragraph.content
print("Content")
#print(content)
print("----------------------------------------")
#split the content into sections
sections = content.split("\n\n")
#remove empty sections
sections = [section for section in sections if section.strip()]
#remove empty lines
sections = [section.replace("\n", " ") for section in sections]

print("Sections")
for section in sections:
    print(section)
    print("----------------------------------------")

Content
----------------------------------------
Sections
 L 65/23 Official Journal of the European Union EN 8.3.2013 COMMISSION DECISION of 7 March 2013 on the safety requirements to be met by European standards for certain seats for children pursuant to Directive 2001/95/EC of the European Parliament and of the Council on general product safety (Text with EEA relevance) (2013/121/EU) THE EUROPEAN COMMISSION, Having regard to the Treaty on the Functioning of the European Union, Having regard to Directive 2001/95/EC of the European Parliament and of the Council of 3 December 2001 on general product safety (1), and in particular Article 4(1)(a) thereof, Whereas: (1) Products which conform to national standards that transpose European standards drawn up under Directive 2001/95/EC and referenced in the Official Journal of the European Union benefit from a presumption of safety. (2) European standards are to be drawn up on the basis of requirements intended to ensure that products which co

In [41]:
#semantic cutting while keepint the single paragraphs intact

import json  
  
def extract_sections_and_paragraphs(result):  
    sections = []  
    current_heading = None  
    current_paragraphs = []  
  
    for i, paragraph in enumerate(result['paragraphs']):  
        if (paragraph.role == "sectionHeading"):  
            if current_heading is not None:  
                sections.append((current_heading, current_paragraphs))  
            current_heading = paragraph['content']  
            current_paragraphs = []  
        else:  
            current_paragraphs.append(paragraph['content'])  
      
    if current_heading is not None:  
        sections.append((current_heading, current_paragraphs))  
      
    return sections  
  
# Extract sections and paragraphs  
sections = extract_sections_and_paragraphs(result)  
  
# Print the result 
for heading, paragraphs in sections:  
    print(f"Heading: {heading}")  
    for paragraph in paragraphs:  
        print(f"  Paragraph: {paragraph}")  

Heading: Article 1
Heading: Definitions
  Paragraph: For the purposes of this Decision, the following definitions apply:
  Paragraph: (a) 'Chair-mounted seat' means a product intended to be fixed onto an adult chair to raise the sitting position of a child up to 36 months old who is able to sit up unaided;
  Paragraph: (b) 'Children's chair' means a chair intended for seating one child, of a size adjusted to the child's age and intended to be placed on the floor;
  Paragraph: (c) 'Children's high chair' means a free-standing chair that elevates the child from six months to 36 months of age to approximately dining-table height, intended for feeding a child who is able to sit up unaided, if the child is appro- priately secured in the sitting position;
  Paragraph: (d) 'Table-mounted chair' means a chair normally used for children who are able to sit up unaided that is intended to be attached to a table or other horizontal surface.
Heading: Article 2 Safety requirements
  Paragraph: The s

In [42]:
#semantic chunking with joining the paragraphs

import json  
  
def extract_sections_and_paragraphs(result):  
    sections = []  
    current_heading = None  
    current_paragraphs = []  
  
    for i, paragraph in enumerate(result['paragraphs']):  
        if (paragraph.role == "sectionHeading" or paragraph.role=="title"):  
            if current_heading is not None:  
                sections.append((current_heading, ' '.join(current_paragraphs)))  
            current_heading = paragraph['content']  
            current_paragraphs = []  
        else:  
            current_paragraphs.append(paragraph['content'])  
      
    if current_heading is not None:  
        sections.append((current_heading, ' '.join(current_paragraphs)))  
      
    return sections  
 
  
# Extract sections and paragraphs  
sections = extract_sections_and_paragraphs(result)  
  
# Print the result  
for heading, paragraph in sections:  
    print(f"Heading: {heading}")  
    print(f"  Paragraph: {paragraph}")  

Heading: COMMISSION DECISION of 7 March 2013
  Paragraph: on the safety requirements to be met by European standards for certain seats for children pursuant to Directive 2001/95/EC of the European Parliament and of the Council on general product safety (Text with EEA relevance) (2013/121/EU) THE EUROPEAN COMMISSION, Having regard to the Treaty on the Functioning of the European Union, Having regard to Directive 2001/95/EC of the European Parliament and of the Council of 3 December 2001 on general product safety (1), and in particular Article 4(1)(a) thereof, Whereas: (1) Products which conform to national standards that transpose European standards drawn up under Directive 2001/95/EC and referenced in the Official Journal of the European Union benefit from a presumption of safety. (2) European standards are to be drawn up on the basis of requirements intended to ensure that products which conform to them satisfy the general safety requirement set out in Article 3 of Directive 2001/95/E

In [43]:
#print the first title of the list of paragraphs
def extract_first_title(result):  
    
      
    for paragraph in result['paragraphs']:  
        if paragraph['role'] == "title":  
            title = paragraph['content']  
            break  
  
    if title:  
        return title 
    else:  
        print("No title found")
        
title= extract_first_title(result)    
print(title)


COMMISSION DECISION of 7 March 2013


In [48]:
def extract_chunks_and_paragraphs(result):  
    chunks = []  
    current_heading = None  
    current_paragraphs = []  
    chunk_id = 1  
  
    for i, paragraph in enumerate(result['paragraphs']):  
        if (paragraph.role == "sectionHeading" or paragraph.role=="title" ):  #take both titles and section headings as sometimes section headings are mistaken as titles
            if current_heading is not None:  
                chunks.append((chunk_id, current_heading, ' '.join(current_paragraphs)))  
                chunk_id += 1  
            current_heading = paragraph['content']  
            current_paragraphs = []  
        else:  
            current_paragraphs.append(paragraph['content'])  
  
    if current_heading is not None:  
        chunks.append((chunk_id, current_heading, ' '.join(current_paragraphs)))  
  
    return chunks  

In [ ]:
import openai
import os

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_key =os.environ["AZURE_OPENAI_API_KEY"]

client = openai.AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2023-09-01-preview"
)

In [51]:
embeddings = client.embeddings.create(
    model="text-embedding-3-large",
    input="The food was delicious and the waiter..."
)
                                
print(embeddings.data[0].embedding)

[-0.010992265306413174, 0.0027546093333512545, -0.00571859534829855, -0.008060994558036327, 0.0006436691037379205, 0.010560426861047745, -0.01123435702174902, -0.056165244430303574, -0.010501539334654808, 0.0394674688577652, -0.004760043695569038, 0.00919293612241745, -0.061766065657138824, 0.07291536033153534, -0.00102807127404958, -0.0033140373416244984, -0.012281239964067936, -0.012333584018051624, -0.036431510001420975, 0.0030359590891748667, -0.021892929449677467, -0.015153623186051846, 0.058363694697618484, -0.016239764168858528, 0.06134730949997902, -0.024497048929333687, -0.012516788206994534, 0.02435310371220112, 0.03742604702711105, 0.015323741361498833, -0.015572376549243927, 0.016737032681703568, 0.03891785442829132, -0.0043870918452739716, -0.02457556501030922, 0.020374950021505356, 0.019301895052194595, -0.011404475197196007, 0.06537780910730362, 0.03245335444808006, -0.05229177698493004, 0.003467798000201583, 0.01468252670019865, -0.01981225050985813, -0.0327150747179985

In [52]:
#create a function to create the embedding for each chunk.
def get_embedding(client, input_chunk):  
    embeddings = client.embeddings.create(  
        model="text-embedding-3-large",  
        input=input_chunk  
    )  
    return embeddings.data[0].embedding  
  

In [53]:
chunks=extract_chunks_and_paragraphs(result)

# test this before pushing into the index

for chunk in chunks:  
    print(f"Chunk ID: {chunk[0]}")  
    print(title)
    print(f"Paragraphs: {chunk[2]}")  
    print( local_file)
    print( local_file) # should be path to the file
    print(f"Heading: {chunk[1]}")  
    #print(get_embedding(client,({"text"+chunk[2]})))
        
    print("---")  

Chunk ID: 1
COMMISSION DECISION of 7 March 2013
Paragraphs: on the safety requirements to be met by European standards for certain seats for children pursuant to Directive 2001/95/EC of the European Parliament and of the Council on general product safety (Text with EEA relevance) (2013/121/EU) THE EUROPEAN COMMISSION, Having regard to the Treaty on the Functioning of the European Union, Having regard to Directive 2001/95/EC of the European Parliament and of the Council of 3 December 2001 on general product safety (1), and in particular Article 4(1)(a) thereof, Whereas: (1) Products which conform to national standards that transpose European standards drawn up under Directive 2001/95/EC and referenced in the Official Journal of the European Union benefit from a presumption of safety. (2) European standards are to be drawn up on the basis of requirements intended to ensure that products which conform to them satisfy the general safety requirement set out in Article 3 of Directive 2001/95

Create vector bases index

In [ ]:
import os
import json

index_name = "demo-euralex-custom-headings-2"

### Create Azure Search Vector-based Index
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': "2024-11-01-preview" }#os.environ['AZURE_SEARCH_API_VERSION']}

In [ ]:
import requests 


# Create the index
index_payload = {
    "name": index_name,
    "vectorSearch": {
        "algorithms": [ 
             {
                 "name": "my-hnsw-config-1",
                 "kind": "hnsw",
                 "hnswParameters": {
                     "m": 4,
                     "efConstruction": 400,
                     "efSearch": 500,
                     "metric": "cosine"
                 }
             }
        ],
        "vectorizers": [
            {
                "name": "openai",
                "kind": "azureOpenAI",
                "azureOpenAIParameters":
                {
                    "resourceUri" : os.environ["AZURE_OPENAI_ENDPOINT"],
                    "apiKey" : os.environ["AZURE_OPENAI_API_KEY"],
                    "deploymentId" : 'text-embedding-3-large',
                    "modelName" : 'text-embedding-3-large'
                }
            }
        ],
        "profiles": [  # profiles is the diferent kind of combinations of algos and vectorizers
            {
             "name": "my-vector-profile-1",
             "algorithm": "my-hnsw-config-1",
             "vectorizer":"openai"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "section_heading"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    },
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "filterable": "true" },
        {"name": "title","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "chunk","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "section_heading","type": "Edm.String","searchable": "false","retrievable": "true"},
        {
            "name": "chunkVector",
            "type": "Collection(Edm.Single)",
            "dimensions": 3072,
            "vectorSearchProfile": "my-vector-profile-1", 
            "searchable": "true",
            "retrievable": "true",
            "filterable": "false",
            "sortable": "false",
            "facetable": "false"
        }
        
    ],
}


azure_search_endpoint= os.environ['AZURE_SEARCH_ENDPOINT']

r = requests.put(azure_search_endpoint+ "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)


print(r.status_code)
print(r.ok)

#print response body of the request
#print(r.json())

204
True


In [56]:
upload_payload = {"value": []}

for chunk in chunks:  
    payload = {
        "@search.action": "upload",
        "id":  str(chunk[0]),
        "title": title,
        "chunk": chunk[2],
        "name": local_file,
        "location": local_file,
        "section_heading": chunk[1],
        "chunkVector": get_embedding(client,("text"+chunk[2]))
        
    }
    #print(payload)
    upload_payload["value"].append(payload)

print("getting payload complete")

r = requests.post(azure_search_endpoint+ "/indexes/" + index_name + "/docs/index",
                    data=json.dumps(upload_payload), headers=headers, params=params)
if r.status_code != 200:
    print(f"Failed to upload: {r.status_code}")
    print(r.text)

getting payload complete


In [ ]:
! pip install azure-search-documents==11.6.0b1 --quiet

In [ ]:
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch
)

azure_search_endpoint= os.environ['AZURE_SEARCH_ENDPOINT']
key= os.environ['AZURE_SEARCH_KEY']
credential = AzureKeyCredential(key)


# Create the Azure Cognitive Search client to issue queries
search_client = SearchClient(
    endpoint=azure_search_endpoint, index_name=index_name, credential=credential
)

In [21]:
QUESTION = "summarize potential safety hazards for children chairs"

In [22]:
# Run an empty query (returns selected fields, all documents)
results =  search_client.search(query_type='simple',
    search_text="*" ,
    select='section_heading,chunk',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["section_heading"])
    #print(f"Description: {result['Description']}")

Total Documents Matching Query: 59
1.0
Surfaces
1.0
2.3. Safety information
1.0
Markings
1.0
Definitions
1.0
Flammable properties
1.0
Safety information
1.0
2.2. Safety requirements
1.0
Structural integrity
1.0
Article 1
1.0
Article 2 Safety requirements
1.0
Article 3 Entry into force
1.0
Identification of the manufacturer and importer
1.0
Ingestion hazards
1.0
Structural integrity
1.0
Purchase information
1.0
Restraint system
1.0
Choking hazards
1.0
Structural integrity
1.0
User manual
1.0
3.1. Scope
1.0
3.3. Safety information, user manual and markings
1.0
User manual
1.0
Falling hazards
1.0
Structural integrity
1.0
4.3. Safety information, user manual and markings
1.0
Purchase information
1.0
GENERAL SAFETY REQUIREMENTS
1.0
Reclinable backrest
1.0
General
1.0
Hazards associated with moving parts
1.0
1.2. Safety requirements
1.0
Surfaces
1.0
Stability
1.0
User manual
1.0
Suffocation hazards
1.0
1.3. Safety information, user manual and markings
1.0
3.2. Safety requirements
1.0
Backres

In [23]:
# Run an empty query (returns selected fields, all documents)
results =  search_client.search(query_type='simple',
    search_text=QUESTION,
    select='section_heading,chunk',
    include_total_count=True,
    top=5)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["section_heading"])
    print(result["chunk"])
    #print(f"Description: {result['Description']}")

Total Documents Matching Query: 42
5.580925
2.1. Scope
These safety requirements apply to children's chairs intended for children who are able to sit up unaided. These include stools, chairs (assembly parts include: legs, seat base and backrest) and armchairs (assembly parts include: seat base, backrest and armrests) for indoor and outdoor use. Rocking chairs and foldable chairs are also included. The requirements also apply to multifunctional products that can be converted into children's chairs. They also apply to children's chairs fitted with wheels. Products that combine the function of a children's chair with another function (for example, storage) must also comply with the requirements.
4.8517833
2.2. Safety requirements
Entrapment hazards from gaps and openings Children's chairs must be designed and manufactured to prevent the entrapment of limbs, feet and hands and as far as possible the entrapment of fingers in gaps and openings. Foldable children's chairs must be designed and

In [ ]:
# Runs a semantic query (runs a BM25-ranked query and promotes the most relevant matches to the top)
results_heading =  search_client.search(query_type='semantic', semantic_configuration_name='my-semantic-config',
    search_text=QUESTION, 
    select='section_heading,chunk',
    filter="filter",
    top=5)


#display the results with line breaks

for result in results_heading:
    print(result["@search.reranker_score"])
    print(result["section_heading"])
    print(result["chunk"])




3.0364677906036377
1.2. Safety requirements
Entrapment hazards from gaps and openings Chair-mounted seats must be designed and manufactured to prevent the entrapment of any part of a child's body. Hazards associated with adjusting the height of the chair-mounted seat Chair-mounted seats in which the height of the sitting area can be adjusted must have locking mechanism(s) to secure the chair-mounted seat in its position of normal use. The unintentional release of locking mechanism(s) must be prevented. Hazards associated with moving parts Once the chair-mounted seat is set up for normal use, there must not be any accessible compression or shear points as a result of moving the chair-mounted seat or any part of it, the child's shifting its body weight while in the chair-mounted seat, or the application of an external force (either by another child or, unintentionally, by the carer, or by a powered mechanism). Chair-mounted seats designed to fold must have a folding mechanism that a chil

In [ ]:
#compare custom index vs wizard answer:


#custom section headings
1.2. Safety requirements
Entrapment hazards from gaps and openings Chair-mounted seats must be designed and manufactured to prevent the entrapment of any part of a child's body. Hazards associated with adjusting the height of the chair-mounted seat Chair-mounted seats in which the height of the sitting area can be adjusted must have locking mechanism(s) to secure the chair-mounted seat in its position of normal use. The unintentional release of locking mechanism(s) must be prevented. Hazards associated with moving parts Once the chair-mounted seat is set up for normal use, there must not be any accessible compression or shear points as a result of moving the chair-mounted seat or any part of it, the child's shifting its body weight while in the chair-mounted seat, or the application of an external force (either by another child or, unintentionally, by the carer, or by a powered mechanism). Chair-mounted seats designed to fold must have a folding mechanism that a child cannot operate and that a carer cannot inadvertently trigger. It must not be possible to set up chair-mounted seats for normal use without activating the locking mechanism. Falling hazards Once it is set up for normal use, the chair-mounted seat must ensure that the child remains in it and that it does not overturn when he or she leans in any direction. To avoid injuries due to the child's standing up and falling or slipping out of the chair-mounted seat, it must be designed in such a way that its restraint system prevents the child from standing up and falling or slipping out of it. The chair-mounted seat must be fitted with a restraint system that can be adjusted to the size of the child and must consist of at least a waist and crotch restraint. It must not be possible to use the restraint system without using the crotch restraint. The restraint system, straps, anchoring points and fastening system must suffer no permanent damage which can impair their safety and normal function when subjected to static and dynamic mechanical stress during reasonable foreseeable use of the chair-mounted seat. When mounted for use, the height of the backrest of the chair-mounted seat must be appropriately high. It must also be fitted with side armrests that are high enough to ensure that a child remains in the seat when he or she leans in any direction.

##################################################################################################################################
# wizard
CELEX_32013D0121_EN_TXT.pdf
requirements apply to children’s chairs intended for children who are able to sit up unaided. These include 
stools, chairs (assembly parts include: legs, seat base and backrest) and armchairs (assembly parts include: seat base, 
backrest and armrests) for indoor and outdoor use. Rocking chairs and foldable chairs are also included. The requirements 
also apply to multifunctional products that can be converted into children’s chairs. They also apply to children’s chairs 
fitted with wheels. Products that combine the function of a children’s chair with another function (for example, storage) 
must also comply with the requirements. 

2.2. Safety requirements 

Entrapment hazards from gaps and openings 

Children’s chairs must be designed and manufactured to prevent the entrapment of limbs, feet and hands and as far as 
possible the entrapment of fingers in gaps and openings. 

Foldable children’s chairs must be designed and manufactured to prevent the entrapment of fingers. 

Children’s chairs must not be so heavy that a child’s head or limb can be entrapped under them. 

Hazards associated with moving parts 

Once the children’s chair is set up for normal use according to the manufacturer’s instructions, there must not be any 
hazardous moving parts. 

Castors and wheels 

Children’s chairs that are fitted with wheels or castors must be designed so as not to impair their stability. 

Falling hazards 

Children’s chairs must be stable enough to prevent overturning in all directions while the child is in them. 

Stability 

Children’s chairs must be stable. 

Choking hazards 

To prevent choking hazards, children’s chairs must not contain small parts that can be detached by forces that a child can 
apply and can fit entirely in a child’s mouth. Stuffing materials that constitute choking hazards must not become 
accessible when submitted to the forces that a child can apply. They must not constitute an additional choking

In [26]:
#compare with standard wizard indexing
# Create the Azure Cognitive Search client to issue queries
search_client_wizard = SearchClient(
    endpoint=azure_search_endpoint, index_name="demo-euralex-wizard", credential=credential
)

In [27]:
# Runs a semantic query (runs a BM25-ranked query and promotes the most relevant matches to the top)
results_wizard =  search_client_wizard.search(query_type='semantic', semantic_configuration_name='demo-euralex-wizard-semantic-configuration',
    search_text=QUESTION, 
    select='title,chunk',
    top=1)

for result in results_wizard:
    print(result["@search.reranker_score"])
    print(result["title"])
    print(result["chunk"])
    #print(f"Description: {result['Description']}")


3.0329229831695557
CELEX_32013D0121_EN_TXT.pdf
requirements apply to children’s chairs intended for children who are able to sit up unaided. These include 
stools, chairs (assembly parts include: legs, seat base and backrest) and armchairs (assembly parts include: seat base, 
backrest and armrests) for indoor and outdoor use. Rocking chairs and foldable chairs are also included. The requirements 
also apply to multifunctional products that can be converted into children’s chairs. They also apply to children’s chairs 
fitted with wheels. Products that combine the function of a children’s chair with another function (for example, storage) 
must also comply with the requirements. 

2.2. Safety requirements 

Entrapment hazards from gaps and openings 

Children’s chairs must be designed and manufactured to prevent the entrapment of limbs, feet and hands and as far as 
possible the entrapment of fingers in gaps and openings. 

Foldable children’s chairs must be designed and manufactured to 